In [1]:
import pandas as pd
import numpy as np
import os
import itertools

In [2]:
os.chdir('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL')
os.getcwd()

'c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL'

In [3]:
scores = pd.read_csv('backend/data/games/scores.csv')
scores['date'] = pd.to_datetime(scores['date'])
scores['first_score_time'] = scores['first_score_time'].apply(lambda x: int(x.split(':')[0]) + round(float(x.split(':')[1]) / 60, 2))
scores['first_score_time'] = scores['first_score_time'] + np.where(scores['first_score_quarter'] == 1, 0, 15)
scores['first_score'] = scores['team'].apply(lambda x: x.split(' ')[-1]) == scores['first_score_team']
scores = scores[['date', 'team', 'opponent', 'week', 'season', 'first_score_team', 'first_score_quarter', 'first_score_time', 'first_score', 'home_field']]
scores.sample(5)

,date,team,opponent,week,season,first_score_team,first_score_quarter,first_score_time,first_score,home_field
6433,2021-11-07,New Orleans Saints,Atlanta Falcons,9,2021,Falcons,1,4.43,False,True
5968,2021-11-28,New York Jets,Houston Texans,12,2021,Jets,1,5.57,True,False
2801,2015-10-25,St. Louis Rams,Cleveland Browns,7,2015,Rams,1,11.98,True,True
459,2010-10-31,Kansas City Chiefs,Buffalo Bills,8,2010,Chiefs,2,16.47,True,True
2112,2013-11-17,Detroit Lions,Pittsburgh Steelers,11,2013,Steelers,1,6.33,False,False


In [4]:
df = scores.groupby(['home_field', 'first_score']).agg({'first_score': 'count'})
df = df.join(scores.groupby(['home_field'])['week'].count(), how='inner')
df['accuracy'] = round(df['first_score'] / df['week'], 2)
df.head(10)

first_score  week  accuracy
home_field first_score                             
False      False               1786  3224      0.55
           True                1438  3224      0.45
True       False               1449  3224      0.45
           True                1775  3224      0.55

In [5]:
df = scores[scores['season'] == 2021].groupby(['team', 'first_score']).agg({'first_score_time': 'mean', 'first_score': 'count'})
df = df.join(scores[scores['season'] == 2021].groupby(['team'])['week'].count(), how='inner')
df['accuracy'] = round(df['first_score'] / df['week'], 2)
df.tail(15)

first_score_time  first_score  week  \
team                     first_score                                        
New York Jets            False                9.469167           12    17   
                         True                 9.200000            5    17   
Philadelphia Eagles      False               10.860909           11    18   
                         True                 8.251429            7    18   
Pittsburgh Steelers      False                8.460000            9    18   
                         True                14.711111            9    18   
San Francisco 49ers      False                9.977500           12    20   
                         True                 7.093750            8    20   
Seattle Seahawks         False               11.628889            9    17   
                         True                10.790000            8    17   
Tampa Bay Buccaneers     False               11.283333            9    19   
                         True                 9.159000           10    19   
Tennessee Titans         False                7.894444            9    18   
                         True                 9.018889            9    18   
Washington Football Team False                9.855882           17    17   

                                      accuracy  
team                     first_score            
New York Jets            False            0.71  
                         True             0.29  
Philadelphia Eagles      False            0.61  
                         True             0.39  
Pittsburgh Steelers      False            0.50  
                         True             0.50  
San Francisco 49ers      False            0.60  
                         True             0.40  
Seattle Seahawks         False            0.53  
                         True             0.47  
Tampa Bay Buccaneers     False            0.47  
                         True             0.53  
Tennessee Titans         False            0.50  
                         True             0.50  
Washington Football Team False            1.00

In [8]:
teams = ('Pittsburgh Steelers', 'Cincinnati Bengals')
first_score = (True, False)
away_first = df.loc[(teams[0], first_score[0]), :]['accuracy'] * df.loc[(teams[1], first_score[1]), :]['accuracy']
home_first = df.loc[(teams[0], first_score[1]), :]['accuracy'] * df.loc[(teams[1], first_score[0]), :]['accuracy']

In [9]:
round(away_first / (away_first + home_first) * 100, 1)

62.0

In [10]:
round(home_first / (away_first + home_first) * 100, 1)

38.0